In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = np.array([1,1,1,1,1,0,0,0,0,0])

In [3]:
one_hot('amazing restaurant',100)

[4, 26]

In [19]:
vocab_size=100

In [20]:
encoded_reviews=[one_hot(d,vocab_size) for d in reviews]
(encoded_reviews)

[[13, 88],
 [5, 41],
 [75, 30],
 [2, 93, 73],
 [26, 75, 16],
 [46, 88],
 [84, 75, 56],
 [71, 8],
 [71, 66],
 [82, 55]]

In [21]:
max_len=3
padded_reviwes=pad_sequences(encoded_reviews,maxlen=max_len,padding="post")
padded_reviwes


array([[13, 88,  0],
       [ 5, 41,  0],
       [75, 30,  0],
       [ 2, 93, 73],
       [26, 75, 16],
       [46, 88,  0],
       [84, 75, 56],
       [71,  8,  0],
       [71, 66,  0],
       [82, 55,  0]])

In [22]:
embeded_vector_size = 4
model=Sequential()
model.add(Embedding(vocab_size,embeded_vector_size,input_length=max_len,name='embedding'))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))

In [23]:
X = padded_reviwes
y = sentiment

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 4)              400       
                                                                 
 flatten_1 (Flatten)         (None, 12)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 13        
                                                                 
Total params: 413
Trainable params: 413
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
model.fit(X, y, epochs=50, verbose=0)


In [26]:
model.evaluate(X,y)

1/1 [==============================] - 0s 226ms/step - loss: 0.6113 - accuracy: 1.0000


[0.611340343952179, 1.0]

In [27]:
 weights=model.get_layer('embedding').get_weights()[0]
len(weights)

100

In [30]:
weights[13]

array([ 0.0420001 , -0.0275777 , -0.09901845,  0.06757111], dtype=float32)

In [31]:
weights[5]

array([ 0.08184871, -0.06241569, -0.03296131,  0.09893958], dtype=float32)

# Gensim

In [32]:
import gensim
import pandas as pd


In [33]:
df=pd.read_json('Cell_Phones_and_Accessories_5.json',lines=True)

In [35]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [36]:
df.shape

(194439, 9)

In [37]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [38]:
review_text=df.reviewText.apply(gensim.utils.simple_preprocess)

In [41]:
review_text


0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [43]:
model=gensim.models.Word2Vec(
           min_count=2,workers=4
)

In [44]:
model.build_vocab(review_text,progress_per=1000)

In [45]:
model.corpus_count

194439

In [46]:
model.epochs

5

In [54]:
model.train(review_text,total_examples=model.corpus_count,epochs=model.epochs)


(61508186, 83868975)

In [55]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")


In [53]:
model.wv.similarity(w1="good",w2="bad")

0.5918173